<a href="https://colab.research.google.com/github/ykim71/georgia_analysis/blob/main/(3)_Georgia_topic_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# need access to the folder(s) below:

#%cd /content/drive/MyDrive/Georgia/


/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia/Locations Analysis


# Topic analysis

## Prep for topic analysis

In [ ]:
%cd /content/drive/MyDrive/Georgia/Topic Dictionaries

import pandas as pd

df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner3_lem.pkl')


/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia/Web Scraping


In [ ]:
# lemmatization if needed
import spacy
import re

nlp = spacy.load("en_core_web_sm", disable=['ner', 'parser']) # disabling Named Entity Recognition for speed


In [ ]:
import re

def get_lemmas_num(text):

    regex = re.compile('[^a-zA-Z0-9]')

    lemmas = []
    
    doc = nlp(text)

    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_ != 'PRON') and (token.text != 'NA'): 
            lemmas.append(regex.sub('', token.lemma_))
    
    return ' '.join(lemmas)

In [ ]:
df['article_lemma_num'] = df['article_final'].str.lower().astype(str).apply(get_lemmas_num)


In [ ]:
df['article_final'][0]
df['article_lemma_num'][0]

'american picker excited return georgia plan film episode history channel hit television series area december 2022  american picker documentary series explore fascinating world antique pick history channel hit follow skilled picker business hunt america valuable antique  excited find sizable unique collection learn interesting story  hit road coast coast picker mission recycle rescue forget relic  way want meet character remarkable exceptional item hope historically significant object new lease life learn thing america past way picker see lot rusty gold year look discover see ready find extraordinary item hear fascinating tale  team american picker continue pandemic seriously follow guideline protocol safe filming outline state cdc excited continue reach collector area discuss year picking eager hear memorable story  american picker tv look lead love explore hidden treasure know large private collection accumulation antique picker spend well day look love hear  note picker pick store f

## word2vec training

In [ ]:
%cd /content/drive/MyDrive/Georgia/Topic Dictionaries

import pandas as pd 

df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner3_lem.pkl') 

[Errno 2] No such file or directory: '/content/drive/MyDrive/Georgia/Topic Dictionaries'
/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia/Web Scraping


FileNotFoundError: ignored

In [ ]:
# To extract n-gram from text
from gensim.models.phrases import Phrases, Phraser
# To train word2vec
from gensim.models import Word2Vec
# To load pre trained word2vec
from gensim.models import KeyedVectors


In [ ]:
df.columns

Index(['index', 'source', 'en_source', 'website', 'article_url',
       'article_title', 'article_author', 'date_published', 'article_final',
       'ner_places', 'ner_orgs', 'article_lemma_num', 'id', 'ner_places_orgs',
       'ner_person'],
      dtype='object')

In [ ]:
sentence_stream = [doc.split(" ") for doc in df['article_lemma_num']]
print(sentence_stream)
print(type(sentence_stream))

[['american', 'picker', 'excited', 'return', 'georgia', 'plan', 'film', 'episode', 'history', 'channel', 'hit', 'television', 'series', 'area', 'december', '2022', '', 'american', 'picker', 'documentary', 'series', 'explore', 'fascinating', 'world', 'antique', 'pick', 'history', 'channel', 'hit', 'follow', 'skilled', 'picker', 'business', 'hunt', 'america', 'valuable', 'antique', '', 'excited', 'find', 'sizable', 'unique', 'collection', 'learn', 'interesting', 'story', '', 'hit', 'road', 'coast', 'coast', 'picker', 'mission', 'recycle', 'rescue', 'forget', 'relic', '', 'way', 'want', 'meet', 'character', 'remarkable', 'exceptional', 'item', 'hope', 'historically', 'significant', 'object', 'new', 'lease', 'life', 'learn', 'thing', 'america', 'past', 'way', 'picker', 'see', 'lot', 'rusty', 'gold', 'year', 'look', 'discover', 'see', 'ready', 'find', 'extraordinary', 'item', 'hear', 'fascinating', 'tale', '', 'team', 'american', 'picker', 'continue', 'pandemic', 'seriously', 'follow', 'gui

In [ ]:
# As Phrases() takes list of list as it's input
# Converting dataframe to list of list

## all lower cases

sentence_stream = [doc.split(" ") for doc in df['article_lemma_num']] 
bigram = Phrases(sentence_stream, min_count= 10, delimiter=b' ') # v2 threshold as default # gensim updates applied; may need to switch some code here 
bigram_phraser = Phraser(bigram)

# bigram = Phrases(sentence_stream, min_count=10, threshold= 10, delimiter=b' ') # higher threshold fewer phrases.
# trigram = Phrases(bigram[sentence_stream], threshold= 100, delimiter=b' ') # include bi- and tri-grams

# Extract bigram for gensim word2vec
bigram_token = []

for sent in sentence_stream:
    bigram_token.append(bigram_phraser[sent])

article_model_bigram = Word2Vec(bigram_token, min_count=10, size= 40, workers=5, window = 5, sg = 0)
# article_model_bigram.save("Georgia_language_lemma_num10_v2") # v2 : with updated dataset


TypeError: ignored

## topic words identification

### separate analysis for words with asterisk

the original list has several words with asterisk as covering repeated phrases

In [ ]:
%cd /content/drive/MyDrive/Georgia/Topic Dictionaries


/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia/Topic Dictionaries


In [ ]:
topics = pd.read_excel("topics.xlsx") # load the original list

topics['word_lemma'] = topics['word'].str.lower().astype(str).apply(get_lemmas_num)


In [ ]:
topics

,topic,word,word_lemma
0,Inflation,Cost of living,cost live
1,Inflation,inflation,inflation
2,Inflation,rising pric*,rise pric
3,Inflation,afford,afford
4,Inflation,affordability,affordability
...,...,...,...
515,Crime,substance abuse,substance abuse
516,Crime,substance use,substance use
517,Crime,weed,weed
518,Crime,cartel,cartel


In [ ]:
import numpy as np

star_phrases = topics['word'].str.contains('\*{1}', case=False, regex=True) 

topics['word_star'] = np.where(star_phrases, 1, 0)


In [ ]:
topics_st = topics[['topic', 'word_lemma', 'word_star']].drop_duplicates()
topics_st.sample(10)

,topic,word_lemma,word_star
114,Immigration / border security,emigrat,1
359,Racial inequality,underrepresented,0
250,Climate change,endanger specie,0
362,Racial inequality,woman color,0
480,Crime,steal car,0
163,Economy / Jobs,lay,0
152,Economy / Jobs,live standard,0
440,Crime,murder,0
38,Health care,hospital,0
92,Health care,surgery,0


In [ ]:
topics_st_asterisk = topics_st[topics_st['word_star']==1]

In [ ]:
topics_st_asterisk_list = topics_st_asterisk['word_lemma'].values.tolist()

In [ ]:
words = pd.DataFrame(list(article_model_bigram.wv.vocab.keys())) #gensim updates; code change needed
words.rename(columns={0:'n_grams'}, inplace=True) 

NameError: ignored

In [ ]:
words['word_lemma'] = words['n_grams'].astype(str).apply(lambda x: [i for i in topics_st_asterisk_list if i in x])


NameError: ignored

In [ ]:
words_filter = words[ words['word_lemma'].str.len() > 0 ] 

In [ ]:
words_filter_unnest = words_filter.explode('word_lemma')


NameError: ignored

In [ ]:
words_filter_unnest = pd.merge(words_filter_unnest, topics_st_asterisk, on="word_lemma", how='left')

In [ ]:
words_filter_unnest.to_excel('additional_topic_words_v2.xlsx')

In [ ]:
# additional_topic_words_v2 = pd.read_excel('additional_topic_words_v2.xlsx') 
# additional_topic_words_v1 = pd.read_excel('additional_topic_words.xlsx')

In [ ]:
# additional_topic_words_v1['version'] = 'v1'
# additional_topic_words_v2['version'] = 'v2'

# add_topic_words_merge = pd.merge( additional_topic_words_v1, additional_topic_words_v2, how="outer", on=["n_grams", "topic"])

In [ ]:
# add_topic_words_merge = add_topic_words_merge[['n_grams', 'topic', 'FINAL', 'version_x', 'version_y']]

In [ ]:
# add_topic_words_merge.to_excel('Topic_words_review1_v1_v2_diff.xlsx')


In [ ]:
#add_topic_words_merge_NA = add_topic_words_merge[ (add_topic_words_merge['version_x'].isnull()) | (add_topic_words_merge['version_y'].isnull()) ]


### final list prep

In [ ]:
add_words = pd.read_excel("Topic_words_review1_v1_v2_diff.xlsx")

In [ ]:
add_words 

,Unnamed: 0,n_grams,topic,FINAL,version_x,version_y,add_from_v2,FINAL2
0,0,2009 abortion,Abortion,0.0,v1,NaN,NaN,0.0
1,1,300000 abortion,Abortion,0.0,v1,NaN,NaN,0.0
2,2,abortion,Abortion,1.0,v1,v2,NaN,1.0
3,3,abortion 2009,Abortion,0.0,v1,v2,NaN,0.0
4,4,abortion access,Abortion,0.0,v1,v2,NaN,0.0
...,...,...,...,...,...,...,...,...
2899,2214,visit httpswwwminorityhealthhhsgov,Racial inequality,0.0,v1,v2,NaN,0.0
2900,2215,vocal minority,Racial inequality,0.0,v1,v2,NaN,0.0
2901,2125,wide blackout,Racial inequality,0.0,v1,v2,NaN,0.0
2902,2867,women minority,Racial inequality,NaN,NaN,v2,NaN,NaN


In [ ]:
topics_st 

,topic,word_lemma,word_star
0,Inflation,cost live,0
1,Inflation,inflation,0
2,Inflation,rise pric,1
3,Inflation,afford,0
4,Inflation,affordability,0
...,...,...,...
513,Crime,pill,0
514,Crime,poison,0
515,Crime,substance abuse,0
516,Crime,substance use,0


In [ ]:
topics_R1 = topics_st[topics_st['word_star'] == 0]
topics_R1 = topics_R1[['topic', 'word_lemma']]
topics_R1

,topic,word_lemma
0,Inflation,cost live
1,Inflation,inflation
3,Inflation,afford
4,Inflation,affordability
5,Inflation,affordable
...,...,...
513,Crime,pill
514,Crime,poison
515,Crime,substance abuse
516,Crime,substance use


In [ ]:
topics_add = add_words[add_words['FINAL2'] == 1]
topics_add = topics_add[['topic', 'n_grams']]
topics_add.rename(columns={'n_grams': 'word_lemma'}, inplace=True)
topics_add

,topic,word_lemma
2,Abortion,abortion
56,Climate change,air quality
59,Climate change,asbestos
78,Climate change,compost
82,Climate change,compostable
...,...,...
2848,Racial inequality,minority
2858,Racial inequality,nondiscrimination
2882,Racial inequality,stigma
2890,Racial inequality,stigmatize


In [ ]:
topic_final = pd.concat([topics_R1, topics_add])

In [ ]:
topic_final = topic_final.drop_duplicates()

In [ ]:
topic_final

,topic,word_lemma
0,Inflation,cost live
1,Inflation,inflation
3,Inflation,afford
4,Inflation,affordability
5,Inflation,affordable
...,...,...
2848,Racial inequality,minority
2858,Racial inequality,nondiscrimination
2882,Racial inequality,stigma
2890,Racial inequality,stigmatize


## topic words similarity

In [ ]:
#%cd /content/drive/MyDrive/Georgia/Topic Dictionaries

import gensim

article_model_bigram = gensim.models.Word2Vec.load("Georgia_language_lemma_num10_v2")


In [ ]:
topic_dictionary = list(set(topic_final.word_lemma.tolist()))

In [ ]:
topic_final # topic covers words in word_lemma; do not remote the duplicated words because topics can share the same words from one another

,topic,word_lemma
0,Inflation,cost live
1,Inflation,inflation
3,Inflation,afford
4,Inflation,affordability
5,Inflation,affordable
...,...,...
2848,Racial inequality,minority
2858,Racial inequality,nondiscrimination
2882,Racial inequality,stigma
2890,Racial inequality,stigmatize


In [ ]:
topic_dictionary #CO: so this is a list of all the ngrams from the column "word_lemma"?

In [ ]:
## update this code to create a dataset with topic words; do it again among the lsd words

word_list_scores = []

for topic_word in topic_dictionary:
  try:
    word_list_scores.append( (topic_word, article_model_bigram.wv.most_similar(topic_word, topn=10)) )
  except KeyError:
    word_list_scores.append( (topic_word, 'NA') )

In [ ]:
df_test = pd.DataFrame ( word_list_scores )
df_test #CO: okay, so we're finding words that are similar to the "topic words"

,0,1
0,lay,"[(bind, 0.7389535307884216), (remove, 0.733186..."
1,right vote,NA
2,recession,"[(downturn, 0.9046533703804016), (stock market..."
3,suspect flee,"[(deputy arrive, 0.8834928274154663), (flee fo..."
4,pill,"[(oral contraceptive, 0.8685505986213684), (pe..."
...,...,...
557,pediatric,"[(american academy, 0.8568011522293091), (onco..."
558,rifle,"[(pistol, 0.8843737840652466), (ar15 style, 0...."
559,inflation,"[(inflationary, 0.8521210551261902), (economy,..."
560,rehire,"[(uvalde school, 0.6184871196746826), (transpo..."


In [ ]:
df_test.rename(columns={0: 'word_lemma', 1: 'similar_words'}, inplace=True)
df_test

,word_lemma,similar_words
0,lay,"[(bind, 0.7389535307884216), (remove, 0.733186..."
1,right vote,NA
2,recession,"[(downturn, 0.9046533703804016), (stock market..."
3,suspect flee,"[(deputy arrive, 0.8834928274154663), (flee fo..."
4,pill,"[(oral contraceptive, 0.8685505986213684), (pe..."
...,...,...
557,pediatric,"[(american academy, 0.8568011522293091), (onco..."
558,rifle,"[(pistol, 0.8843737840652466), (ar15 style, 0...."
559,inflation,"[(inflationary, 0.8521210551261902), (economy,..."
560,rehire,"[(uvalde school, 0.6184871196746826), (transpo..."


In [ ]:
df_test.rename(columns={0: 'word_lemma', 1: 'similar_words'}, inplace=True)

df_text_spread = df_test.explode('similar_words')
df_text_spread

,word_lemma,similar_words
0,lay,"(bind, 0.7389535307884216)"
0,lay,"(remove, 0.7331869602203369)"
0,lay,"(clear, 0.7160282731056213)"
0,lay,"(stand, 0.6987937688827515)"
0,lay,"(retain, 0.6617229580879211)"
...,...,...
561,pharmacy,"(walmart, 0.7072194814682007)"
561,pharmacy,"(cvs, 0.7048928737640381)"
561,pharmacy,"(pharmacy fitness, 0.6944006085395813)"
561,pharmacy,"(doordash, 0.6855670213699341)"


In [ ]:
# df_text_spread_NA = df_text_spread[df_text_spread['similar_words']=='NA']
# df_text_spread_NA.to_excel('FINA_topic_words_not_include_review.xlsx')

In [ ]:
df_text_spread = df_text_spread[df_text_spread['similar_words']!='NA']
df_text_spread

,word_lemma,similar_words
0,lay,"(bind, 0.7389535307884216)"
0,lay,"(remove, 0.7331869602203369)"
0,lay,"(clear, 0.7160282731056213)"
0,lay,"(stand, 0.6987937688827515)"
0,lay,"(retain, 0.6617229580879211)"
...,...,...
561,pharmacy,"(walmart, 0.7072194814682007)"
561,pharmacy,"(cvs, 0.7048928737640381)"
561,pharmacy,"(pharmacy fitness, 0.6944006085395813)"
561,pharmacy,"(doordash, 0.6855670213699341)"


In [ ]:
df_text_spread['similar_word'], df_text_spread['similarity'] = df_text_spread.similar_words.str[0], df_text_spread.similar_words.str[1]
df_text_spread #CO: I'm getting a warning message here. Are you, too? ("SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame. Try using .loc[row_indexer,col_indexer] = value instead")

<ipython-input-46-725813e085df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text_spread['similar_word'], df_text_spread['similarity'] = df_text_spread.similar_words.str[0], df_text_spread.similar_words.str[1]
<ipython-input-46-725813e085df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text_spread['similar_word'], df_text_spread['similarity'] = df_text_spread.similar_words.str[0], df_text_spread.similar_words.str[1]


,word_lemma,similar_words,similar_word,similarity
0,lay,"(bind, 0.7389535307884216)",bind,0.738954
0,lay,"(remove, 0.7331869602203369)",remove,0.733187
0,lay,"(clear, 0.7160282731056213)",clear,0.716028
0,lay,"(stand, 0.6987937688827515)",stand,0.698794
0,lay,"(retain, 0.6617229580879211)",retain,0.661723
...,...,...,...,...
561,pharmacy,"(walmart, 0.7072194814682007)",walmart,0.707219
561,pharmacy,"(cvs, 0.7048928737640381)",cvs,0.704893
561,pharmacy,"(pharmacy fitness, 0.6944006085395813)",pharmacy fitness,0.694401
561,pharmacy,"(doordash, 0.6855670213699341)",doordash,0.685567


In [ ]:
df_text_spread = df_text_spread[df_text_spread['similar_words']!='NA']
df_text_spread['similar_word'], df_text_spread['similarity'] = df_text_spread.similar_words.str[0], df_text_spread.similar_words.str[1]
df_text_spread = df_text_spread.drop('similar_words', axis=1)
df_text_spread

,word_lemma,similar_word,similarity
0,lay,bind,0.738954
0,lay,remove,0.733187
0,lay,clear,0.716028
0,lay,stand,0.698794
0,lay,retain,0.661723
...,...,...,...
561,pharmacy,walmart,0.707219
561,pharmacy,cvs,0.704893
561,pharmacy,pharmacy fitness,0.694401
561,pharmacy,doordash,0.685567


In [ ]:
topic_final

,topic,word_lemma
0,Inflation,cost live
1,Inflation,inflation
3,Inflation,afford
4,Inflation,affordability
5,Inflation,affordable
...,...,...
2848,Racial inequality,minority
2858,Racial inequality,nondiscrimination
2882,Racial inequality,stigma
2890,Racial inequality,stigmatize


In [ ]:
df_text_spread_merge = pd.merge(df_text_spread, topic_final, how='left', on='word_lemma')
df_text_spread_merge

,word_lemma,similar_word,similarity,topic
0,lay,bind,0.738954,Economy / Jobs
1,lay,remove,0.733187,Economy / Jobs
2,lay,clear,0.716028,Economy / Jobs
3,lay,stand,0.698794,Economy / Jobs
4,lay,retain,0.661723,Economy / Jobs
...,...,...,...,...
5105,pharmacy,walmart,0.707219,Health care
5106,pharmacy,cvs,0.704893,Health care
5107,pharmacy,pharmacy fitness,0.694401,Health care
5108,pharmacy,doordash,0.685567,Health care


In [ ]:
# those with higher than 0.7 similarity

df_test_sim07 = df_text_spread_merge[ df_text_spread_merge['similarity'] > 0.7  ]
df_test_sim07

,word_lemma,similar_word,similarity,topic
0,lay,bind,0.738954,Economy / Jobs
1,lay,remove,0.733187,Economy / Jobs
2,lay,clear,0.716028,Economy / Jobs
10,recession,downturn,0.904653,Economy / Jobs
11,recession,stock market,0.832507,Economy / Jobs
...,...,...,...,...
5102,pharmacy,pharmacist refuse,0.733437,Health care
5103,pharmacy,clinic,0.715371,Health care
5104,pharmacy,fill prescription,0.710651,Health care
5105,pharmacy,walmart,0.707219,Health care


In [ ]:
# 4681 previous
# 4391 current

In [ ]:
df_test_sim07 # 4681

,word_lemma,similar_word,similarity,topic
0,lay,bind,0.738954,Economy / Jobs
1,lay,remove,0.733187,Economy / Jobs
2,lay,clear,0.716028,Economy / Jobs
10,recession,downturn,0.904653,Economy / Jobs
11,recession,stock market,0.832507,Economy / Jobs
...,...,...,...,...
5102,pharmacy,pharmacist refuse,0.733437,Health care
5103,pharmacy,clinic,0.715371,Health care
5104,pharmacy,fill prescription,0.710651,Health care
5105,pharmacy,walmart,0.707219,Health care


In [ ]:
df_test_sim07_black = df_test_sim07[df_test_sim07['word_lemma']=='black']
df_test_sim07_black

,word_lemma,similar_word,similarity,topic
720,black,white,0.844675,Racial inequality
721,black,color,0.715145,Racial inequality


In [ ]:
df_test_sim07_nondup = df_test_sim07[['similar_word', 'topic']].drop_duplicates() #CO: Will there be a final validation of the dictionary, to check how much noise there is? Like, a word like "color" could mean many other things, even if the similarity score is high.

In [ ]:
df_test_sim07_nondup = df_test_sim07.drop_duplicates(subset=['similar_word', 'topic']) #CO: are we sure we want "topic in here, too?" Why not only drop rows where "similar_word" is a duplicate?

In [ ]:
df_test_sim07_nondup # 3289 non-duplicates --> 3145

,word_lemma,similar_word,similarity,topic
0,lay,bind,0.738954,Economy / Jobs
1,lay,remove,0.733187,Economy / Jobs
2,lay,clear,0.716028,Economy / Jobs
10,recession,downturn,0.904653,Economy / Jobs
11,recession,stock market,0.832507,Economy / Jobs
...,...,...,...,...
5100,pharmacy,walgreen,0.811342,Health care
5102,pharmacy,pharmacist refuse,0.733437,Health care
5104,pharmacy,fill prescription,0.710651,Health care
5105,pharmacy,walmart,0.707219,Health care


In [ ]:
topic_dictionary

In [ ]:
# exclude those from the topic dictionary #CO: What exactly is being exluced here? Why?
df_test_sim07_final = df_test_sim07_nondup[~df_test_sim07_nondup['similar_word'].isin(topic_dictionary)] #CO: Oh, so is it because we want to exclude words from the "similar_word" column IF the word was already listed in the topic dictionary to begin with?
df_test_sim07_final

,word_lemma,similar_word,similarity,topic
0,lay,bind,0.738954,Economy / Jobs
1,lay,remove,0.733187,Economy / Jobs
2,lay,clear,0.716028,Economy / Jobs
10,recession,downturn,0.904653,Economy / Jobs
11,recession,stock market,0.832507,Economy / Jobs
...,...,...,...,...
5100,pharmacy,walgreen,0.811342,Health care
5102,pharmacy,pharmacist refuse,0.733437,Health care
5104,pharmacy,fill prescription,0.710651,Health care
5105,pharmacy,walmart,0.707219,Health care


In [ ]:
#df_test_sim07_final.to_excel('df_test_sim07_top10_no_duplicates2_v2.xlsx') #CO: only one is being saved here

In [ ]:
#df_test_sim07_top10_v1 = pd.read_excel('df_test_sim07_top10_no_duplicates2.xlsx') #... CO: but a version1 and a version 2 is being loaded here. Why? What's the point of the two versions?
#df_test_sim07_top10_v2 = pd.read_excel('df_test_sim07_top10_no_duplicates2_v2.xlsx') #CO: Also, what is the purpose of these two new versions?



In [ ]:
df_test_sim07_top10_v2 = df_test_sim07_final #CO: doing this avoid saving/re-loading
df_test_sim07_top10_v2

,word_lemma,similar_word,similarity,topic
0,lay,bind,0.738954,Economy / Jobs
1,lay,remove,0.733187,Economy / Jobs
2,lay,clear,0.716028,Economy / Jobs
10,recession,downturn,0.904653,Economy / Jobs
11,recession,stock market,0.832507,Economy / Jobs
...,...,...,...,...
5100,pharmacy,walgreen,0.811342,Health care
5102,pharmacy,pharmacist refuse,0.733437,Health care
5104,pharmacy,fill prescription,0.710651,Health care
5105,pharmacy,walmart,0.707219,Health care


In [ ]:
df_test_sim07_top10_v1 = df_test_sim07_top10_v1[['similar_word', 'topic', 'FINAL']]

In [ ]:
df_test_sim07_top10_v2 = df_test_sim07_top10_v2[['similar_word', 'topic']]
df_test_sim07_top10_v2

,similar_word,topic
0,bind,Economy / Jobs
1,remove,Economy / Jobs
2,clear,Economy / Jobs
10,downturn,Economy / Jobs
11,stock market,Economy / Jobs
...,...,...
5100,walgreen,Health care
5102,pharmacist refuse,Health care
5104,fill prescription,Health care
5105,walmart,Health care


In [ ]:
df_test_sim07_top10_v2['version'] = 'v2'
df_test_sim07_top10_v2

,similar_word,topic,version
0,bind,Economy / Jobs,v2
1,remove,Economy / Jobs,v2
2,clear,Economy / Jobs,v2
10,downturn,Economy / Jobs,v2
11,stock market,Economy / Jobs,v2
...,...,...,...
5100,walgreen,Health care,v2
5102,pharmacist refuse,Health care,v2
5104,fill prescription,Health care,v2
5105,walmart,Health care,v2


In [ ]:
df_sim07_top10_merge = pd.merge( df_test_sim07_top10_v1, df_test_sim07_top10_v2, how="outer", on=["similar_word", "topic"]) #CO: I can't run this

NameError: ignored

In [ ]:
df_sim07_top10_merge[df_sim07_top10_merge['version'].isnull()]

NameError: ignored

In [ ]:
df_sim07_top10_merge[df_sim07_top10_merge['version']=='v2']

NameError: ignored

In [ ]:
#df_sim07_top10_merge.to_excel('df_sim07_top10_merge_v1_v2_diff.xlsx')

In [ ]:
topic_final_r1 = topic_final.copy()
topic_final_r1['topic_final'] = 1
topic_final_r1

,topic,word_lemma,topic_final
0,Inflation,cost live,1
1,Inflation,inflation,1
3,Inflation,afford,1
4,Inflation,affordability,1
5,Inflation,affordable,1
...,...,...,...
2848,Racial inequality,minority,1
2858,Racial inequality,nondiscrimination,1
2882,Racial inequality,stigma,1
2890,Racial inequality,stigmatize,1


In [ ]:
topic_final_r1.rename(columns={'word_lemma': 'similar_word'}, inplace=True) #CO: Is this renaming correct? Didn't "word_lemma" and "similar_words" have different meanings above?


In [ ]:
df_sim07_top10_merge_all = pd.merge(df_sim07_top10_merge, topic_final_r1, how="outer", on=["similar_word", "topic"]) #CO: is it sometimes better to merge based on an ID column? #CO: Also, I'm getting error messages here.

NameError: ignored

In [ ]:
df_sim07_top10_merge_all['review_done'] = np.where(  (df_sim07_top10_merge_all['FINAL']== 0) & (df_sim07_top10_merge_all['version']== "v2"), 1, 0  )


NameError: ignored

In [ ]:
df_sim07_top10_merge_all.to_excel('df_sim07_top10_merge_v1_v2_diff.xlsx')

## final topic word list

In [ ]:
import pandas as pd
topic_R2 = pd.read_excel('df_test_sim07_top10_no_duplicates2.xlsx')

In [ ]:
topic_R2 = topic_R2[topic_R2['FINAL'] == 1] #CO: I'm not sure what this FINAL columns refers to. I think it came from a sheet that was loaded but I didn't see how it got made.
topic_R2 = topic_R2[['topic', 'similar_word']]
topic_R2.rename(columns={'similar_word': 'word_lemma'}, inplace=True)


In [ ]:
topic_R2_add = pd.read_excel('additional_words_manual.xlsx') #CO: I'm not finding this file
topic_R2_add.rename(columns={'similar_word': 'word_lemma'}, inplace=True)
topic_R2_add

FileNotFoundError: ignored

In [ ]:
topic_final

,topic,word_lemma
0,Inflation,cost live
1,Inflation,inflation
3,Inflation,afford
4,Inflation,affordability
5,Inflation,affordable
...,...,...
2848,Racial inequality,minority
2858,Racial inequality,nondiscrimination
2882,Racial inequality,stigma
2890,Racial inequality,stigmatize


In [ ]:
topic_final[topic_final['word_lemma']=='undocumente']

,topic,word_lemma


In [ ]:
topic_final2 = pd.concat([topic_final, topic_R2, topic_R2_add]) #CO: I can't run this

NameError: ignored

In [ ]:
topic_final2

NameError: ignored

In [ ]:
len(topic_final2)

940

In [ ]:
topic_final2 = topic_final2.drop_duplicates().sort_values(['topic','word_lemma']).reset_index()

In [ ]:
topic_final2 = topic_final2[['topic','word_lemma']]

In [ ]:
# delete depression AND lay from Economy / Jobs	 #CO: Why delete these? -> from manual comments 
# delete traffic from Immigration / border security AND Crime

In [ ]:
topic_final2[topic_final2['word_lemma'] == 'undocumented'] # 467


,topic,word_lemma
821,Immigration / border security,undocumented


In [ ]:
topic_final2[topic_final2['word_lemma'] == 'depression'] # 418
topic_final2[topic_final2['word_lemma'] == 'lay'] # 467
topic_final2[topic_final2['word_lemma'] == 'traffic'] # 371, 818
topic_final2[topic_final2['word_lemma'] == 'undocumente'] # 820

,topic,word_lemma
820,Immigration / border security,undocumente


In [ ]:
topic_final2[topic_final2['word_lemma'] == 'cap trade '] # 94

,topic,word_lemma
94,Climate change,cap trade


In [ ]:
topic_final2.drop([94, 418,467, 371, 818, 820], axis=0, inplace=True) #CO: Recommend double checking that this is still correct. When updating  to a new file (as I think you mentioned), I can easily seeing something go wrong here, if the indices change to something else b/c a new file is used.


In [ ]:
#topic_final2.to_excel('FINAL_topic_words_R2_review.xlsx')

## mturk review for v2

this data came from the mturk review separate script

In [ ]:
#%cd /content/drive/MyDrive/Georgia/Topic Dictionaries

review_v2 = pd.read_excel('FINAL_sim07_top10_merge.xlsx') #CO: Only editing here to be able to load the file on my computer. Nothing wrong
review_v2

,Unnamed: 0,similar_word,topic,version,TJS,FINAL_ADD
0,3240,access reproductive,Abortion,v2,1,1.0
1,2988,bodily autonomy,Abortion,v2,1,1.0
2,3985,surgery repair,Health care,v2,(*should have surgery),0.0
3,3127,smuggling fentanyl,Crime,v2,(*should have smuggling already),0.0
4,4148,smuggle burmese,Crime,v2,(*should have smuggle already),0.0
...,...,...,...,...,...,...
5134,2893,dobb,Supreme Court,NaN,NaN,NaN
5135,2894,justice,Supreme Court,NaN,NaN,NaN
5136,2895,ruling,Supreme Court,v2,NaN,NaN
5137,2896,us supreme,Supreme Court,v2,NaN,NaN


In [ ]:
review_v2_review = review_v2[ (review_v2['FINAL_ADD']==1)]

In [ ]:
review_v2_review_st = review_v2_review[['similar_word', 'topic']]

In [ ]:
len(review_v2_review_st)

317

## find example sentences

In [ ]:
import pandas as pd #CO: already imported!

#df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl')

#%cd /content/drive/MyDrive/Georgia/Topic Dictionaries


df = pd.read_pickle('Georgia_data_v2_translation_ner3_lem.pkl') #CO: I assume this is the file we want. But recommend double checking that the correct final file is being used here.

/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia/Web Scraping


In [ ]:
len(df[df['article_lemma_num'].isnull()])

0

In [ ]:
df

### round 1

In [ ]:
topic_final2 = pd.read_excel('FINAL_topic_words_R2_review.xlsx')
topic_final2

/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia/Topic Dictionaries


,Unnamed: 0.1,Unnamed: 0,topic,word_lemma
0,0,0,Abortion,aborted
1,1,1,Abortion,abortion
2,2,2,Abortion,anti abortion
3,3,3,Abortion,anti choice
4,4,4,Abortion,antichoice
...,...,...,...,...
928,928,934,Racial inequality,underrepresented
929,929,935,Racial inequality,white supremacist
930,930,936,Racial inequality,white supremacy
931,931,937,Racial inequality,woman color


In [ ]:
topic_final2.rename(columns={'word_lemma': 'topic_ng'}, inplace=True)
topic_final2 #CO: two of the ID cols don't correspond. Need to be careful, maybe, later on?

,Unnamed: 0.1,Unnamed: 0,topic,topic_ng
0,0,0,Abortion,aborted
1,1,1,Abortion,abortion
2,2,2,Abortion,anti abortion
3,3,3,Abortion,anti choice
4,4,4,Abortion,antichoice
...,...,...,...,...
928,928,934,Racial inequality,underrepresented
929,929,935,Racial inequality,white supremacist
930,930,936,Racial inequality,white supremacy
931,931,937,Racial inequality,woman color


In [ ]:
topic_final2 = topic_final2[['topic','topic_ng']].drop_duplicates() # 933
topic_final2 #CO: OK, maybe you're just always/consistently using the ID col that's left here? If that one is always used, then that should prevent problems!

,topic,topic_ng
0,Abortion,aborted
1,Abortion,abortion
2,Abortion,anti abortion
3,Abortion,anti choice
4,Abortion,antichoice
...,...,...
928,Racial inequality,underrepresented
929,Racial inequality,white supremacist
930,Racial inequality,white supremacy
931,Racial inequality,woman color


In [ ]:
topic_final2 = topic_final2[topic_final2['topic_ng'] != 'choice'] # remove choice due to noise

### round 2

In [ ]:
#CO: I get a warning here: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
review_v2_review_st = review_v2_review[['similar_word', 'topic']]
review_v2_review_st.rename(columns={'similar_word': 'topic_ng'}, inplace=True)
review_v2_review_st = review_v2_review_st[['topic','topic_ng']].drop_duplicates() # 313
review_v2_review_st

<ipython-input-99-0927373e6f76>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_v2_review_st.rename(columns={'similar_word': 'topic_ng'}, inplace=True)


,topic,topic_ng
0,Abortion,access reproductive
1,Abortion,bodily autonomy
5,Abortion,brittany fonteno
9,Abortion,fetal
10,Abortion,life unborn
...,...,...
1664,Racial inequality,practice disproportionately
2004,Racial inequality,prejudicial
2140,Racial inequality,protest racial
4255,Racial inequality,racial disparity


In [ ]:
len(review_v2_review_st)

314

In [ ]:
topic_words = list(set(review_v2_review_st['topic_ng'].tolist())) # make sure change this

word_bigrams_done = list(set(bigrams(topic_words)))
print(word_bigrams_done)
print(len(word_bigrams_done))
                       
word_unigrams_done = unigrams(topic_words)
print(word_unigrams_done)
print(len(word_unigrams_done))

['assault possession', 'surgical procedure', 'housing affordability', 'enroll medicare', 'equity diversity', 'primary caregiver', 'wealth gap', 'brain damage', 'price soar', 'monoclonal antibody', 'brittany fonteno', 'drug smuggling', 'practice disproportionately', 'death penalty', 'raise interest', 'drug addict', 'kindergarten class', 'autoimmune disease', 'mri scan', 'burglary suspect', 'inflation fed', 'wildfire smoke', 'perform cpr', 'migrant cross', 'drug alcohol', 'chronic illness', 'coal fire', 'suicide bombing', 'fed raise', 'american academy', 'oath allegiance', 'life unborn', 'shortage affordable', 'mobile clinic', 'qualified staffing', 'arrest allegedly', 'educational institution', 'hike interest', 'stock bond', 'abbott bus', 'borrower creditworthiness', 'goldman sachs', 'threat democracy', 'indecent exposure', 'pay upfront', 'ivy league', 'coral reef', 'income household', 'growth slow', 'border security', 'taxable income', 'vulnerable specie', 'cancer diagnosis', 'bodily au

In [ ]:

df['topic_ng1'] = df['article_lemma_num'].astype(str).apply(lambda x: set.intersection(set(x.split(' ')), word_unigrams_done))
df['topic_ng2'] = df['article_lemma_num'].astype(str).apply(lambda x: [i for i in word_bigrams_done if i in x])


<ipython-input-103-b95b670edd42>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['topic_ng1'] = df['article_lemma_num'].astype(str).apply(lambda x: set.intersection(set(x.split(' ')), word_unigrams_done))
<ipython-input-103-b95b670edd42>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['topic_ng2'] = df['article_lemma_num'].astype(str).apply(lambda x: [i for i in word_bigrams_done if i in x])


In [ ]:
df.sample(3) #CO: so here the colums "topic_ng1/2" are being completed without the actual topic dictinoary. I assume that dictionary will then be merged/used later

,index,source,en_source,website,article_url,article_title,article_author,date_published,article_final,ner_places,ner_orgs,article_lemma_num,id,ner_places_orgs,ner_person,topic_ng1,topic_ng2
88,88,Metter Advertiser,1.0,http://www.metteradvertiser.com/,http://www.metteradvertiser.com/online_feature...,Help Discover the Next Breakthrough Medical Tr...,NaN,2022-10-10,"(Family Features) Without clinical trials, the...",{'the United States (GPE)'},"{'Getty Images (ORG)', 'AKF (ORG)', 'the Ameri...",family feature clinical trial medicine million...,89,"{Black (LOC), Getty Images (ORG), AKF (ORG), t...",{LaVarne Burton (PERSON)},{},[]
95,95,Metter Advertiser,1.0,http://www.metteradvertiser.com/,http://www.metteradvertiser.com/online_feature...,Resources for Selecting a Health Care Provider,NaN,2022-10-10,(Family Features) Getting health care for your...,{'Visit Medicare.gov/care-compare (LOC)'},{'the U.S. Department of Health & Human Servic...,family feature get health care loved big respo...,96,"{A Resource for Research\nThe (ORG), Getty Ima...",{},"{nursing, hospital}",[]
86,86,Metter Advertiser,1.0,http://www.metteradvertiser.com/,http://www.metteradvertiser.com/online_feature...,Understanding AFib: Living with and treating a...,NaN,2022-09-23,(Family Features) A diagnosis of atrial fibril...,{'the United States (GPE)'},"{'the Heart Rhythm Society (ORG)', 'AFib (ORG)...",family feature diagnosis atrial fibrillation a...,87,"{Biosense Webster, Inc. (ORG), the American He...",{},{prognosis},[atrial fibrillation]


In [ ]:
# CO: I think this is a repetition of something that happened above. Maybe there's a whole block of codes down here that can be deleted? If you make sure that the object names aren't being used for other things then this DF (and what was done with it) should still be available in memory down here, right?
text_df_uni = df[(df['topic_ng1'].str.len()) > 0]
text_df_bi = df[(df['topic_ng2'].str.len()) > 0]

text_df_uni = text_df_uni[['id', 'source', 'article_final', 'topic_ng1']]
text_df_bi = text_df_bi[['id', 'source','article_final', 'topic_ng2']]

text_df_uni_df = text_df_uni.explode('topic_ng1')
text_df_bi_df = text_df_bi.explode('topic_ng2')

text_df_uni_df = text_df_uni_df.rename(columns={'topic_ng1': 'topic_ng'})
text_df_bi_df = text_df_bi_df.rename(columns={'topic_ng2': 'topic_ng'})

text_df_ng_all = pd.concat([text_df_uni_df, text_df_bi_df])



In [ ]:
text_df_ng_all = pd.merge(text_df_ng_all, review_v2_review_st, how='left', on = 'topic_ng')

In [ ]:
text_df_ng_all # three words missing #CO: which three words are missing?

,id,source,article_final,topic_ng,topic
0,1,Metter Advertiser,The American Pickers are excited to return to ...,business,Economy / Jobs
1,12,Metter Advertiser,Candler County remains under a Tropical Storm ...,hospital,Health care
2,12,Metter Advertiser,Candler County remains under a Tropical Storm ...,funding,Economy / Jobs
3,12,Metter Advertiser,Candler County remains under a Tropical Storm ...,business,Economy / Jobs
4,13,Metter Advertiser,Our 2023 Candler County Community Guide has a ...,business,Economy / Jobs
...,...,...,...,...,...
56,75,Metter Advertiser,(Family Features) Sending kids off to college ...,anxiety depression,Health care
57,80,Metter Advertiser,(Family Features) There’s nothing cuter than t...,breast milk,Health care
58,81,Metter Advertiser,(Family Features) For first-time parents or ev...,american academy,Health care
59,87,Metter Advertiser,(Family Features) A diagnosis of atrial fibril...,atrial fibrillation,Health care


In [ ]:
temp_count = text_df_ng_all.groupby(['topic', 'topic_ng']).count().reset_index()

In [ ]:
temp_count

,topic,topic_ng,id,source,article_final
0,Affordable housing,developer,1,1,1
1,Affordable housing,poverty,1,1,1
2,Climate change,agriculture,1,1,1
3,Climate change,carbon,1,1,1
4,Crime,gang relate,1,1,1
5,Economy / Jobs,business,13,13,13
6,Economy / Jobs,funding,4,4,4
7,Economy / Jobs,inflation,1,1,1
8,Economy / Jobs,wealth gap,1,1,1
9,Education,educational institution,1,1,1


In [ ]:
text_df_ng_all

,id,source,article_final,topic_ng,topic
0,1,Metter Advertiser,The American Pickers are excited to return to ...,business,Economy / Jobs
1,12,Metter Advertiser,Candler County remains under a Tropical Storm ...,hospital,Health care
2,12,Metter Advertiser,Candler County remains under a Tropical Storm ...,funding,Economy / Jobs
3,12,Metter Advertiser,Candler County remains under a Tropical Storm ...,business,Economy / Jobs
4,13,Metter Advertiser,Our 2023 Candler County Community Guide has a ...,business,Economy / Jobs
...,...,...,...,...,...
56,75,Metter Advertiser,(Family Features) Sending kids off to college ...,anxiety depression,Health care
57,80,Metter Advertiser,(Family Features) There’s nothing cuter than t...,breast milk,Health care
58,81,Metter Advertiser,(Family Features) For first-time parents or ev...,american academy,Health care
59,87,Metter Advertiser,(Family Features) A diagnosis of atrial fibril...,atrial fibrillation,Health care


In [ ]:
# just include those with less than 10 to make easier sample extraction (replace = True)
merge_uni_both = text_df_ng_all.groupby(['topic', 'topic_ng']).apply(lambda x: x.sample(10, replace=True)).reset_index(drop=True) #CO: what is the purpose of "sample" here? Why take 10? Also, shouldn't "replace" be FALSE?


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
by_sentence = merge_uni_both['article_final'].apply(lambda x : sent_tokenize(x)).apply(pd.Series,1).stack()
by_sentence_df = pd.DataFrame(by_sentence)
by_sentence_df.rename(columns={0:'sentence_each'}, inplace=True)

by_sentence_df['sentence_turn'] = by_sentence_df.index.get_level_values(1)
by_sentence_df.index = by_sentence_df.index.get_level_values(0)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
by_sentence_df.head(30) #CO: I stopped a "\n" in row 23. Should that be cleaned out?
#CO: Is the ID column then being used to keep track of which article this was?

,sentence_each,sentence_turn
0,(Family Features) Many consumers are curious t...,0
0,“GMO” is a common term used to describe foods ...,1
0,Genetic engineering allows scientists to copy ...,2
0,Purpose of GMOs\nHumans have modified crops an...,3
0,"Crossbreeding, selective breeding and mutation...",4
0,These methods often involve mixing all the gen...,5
0,"For example, today’s strawberries are a cross ...",6
0,Modern technology allows scientists to use gen...,7
0,The reasons for genetic modification today are...,8
0,Some GMO plants have been modified to improve ...,9


In [ ]:
merge_all_join = pd.concat([merge_uni_both, by_sentence_df], axis=1)
merge_all_join['sentence_each_lemma'] = merge_all_join['sentence_each'].str.lower().astype(str).apply(get_lemmas_num)
#CO: Why do we need to break this into the sentence level, not just keep it at the article level?


In [ ]:
merge_all_join.head(50) #CO: looks correct! article_final, sentence_each, sentence_each_lemma seem to fit together as they should

,id,source,article_final,topic_ng,topic,sentence_each,sentence_turn,sentence_each_lemma
0,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,(Family Features) Many consumers are curious t...,0,family feature consumer curious learn purpose ...
0,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,“GMO” is a common term used to describe foods ...,1,gmo common term describe food organism plant m...
0,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,Genetic engineering allows scientists to copy ...,2,genetic engineering allow scientist copy gene ...
0,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,Purpose of GMOs\nHumans have modified crops an...,3,purpose gmos human modify crop animal suit ne...
0,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,"Crossbreeding, selective breeding and mutation...",4,crossbreed selective breeding mutation breedin...
0,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,These methods often involve mixing all the gen...,5,method involve mix gene different source
0,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,"For example, today’s strawberries are a cross ...",6,example today strawberry cross strawberry spec...
0,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,Modern technology allows scientists to use gen...,7,modern technology allow scientist use genetic ...
0,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,The reasons for genetic modification today are...,8,reason genetic modification today similar thou...
0,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,Some GMO plants have been modified to improve ...,9,gmo plant modify improve nutritional value


In [ ]:
# find only sentences containing topic_ng word; to extract sentence level examples 
merge_all_join['topic_words_YN'] = merge_all_join.apply(lambda x: x.topic_ng in x.sentence_each_lemma, axis=1)
merge_all_join['topic_words_YN_lead'] = merge_all_join.groupby(['topic', 'topic_ng'])['topic_words_YN'].shift(-1)
merge_all_join.reset_index(drop=True, inplace=True)
merge_all_join.head(50) 

,id,source,article_final,topic_ng,topic,sentence_each,sentence_turn,sentence_each_lemma,topic_words_YN,topic_words_YN_lead
0,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,(Family Features) Many consumers are curious t...,0,family feature consumer curious learn purpose ...,False,False
1,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,“GMO” is a common term used to describe foods ...,1,gmo common term describe food organism plant m...,False,False
2,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,Genetic engineering allows scientists to copy ...,2,genetic engineering allow scientist copy gene ...,False,False
3,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,Purpose of GMOs\nHumans have modified crops an...,3,purpose gmos human modify crop animal suit ne...,False,False
4,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,"Crossbreeding, selective breeding and mutation...",4,crossbreed selective breeding mutation breedin...,False,False
5,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,These methods often involve mixing all the gen...,5,method involve mix gene different source,False,False
6,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,"For example, today’s strawberries are a cross ...",6,example today strawberry cross strawberry spec...,False,False
7,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,Modern technology allows scientists to use gen...,7,modern technology allow scientist use genetic ...,False,False
8,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,The reasons for genetic modification today are...,8,reason genetic modification today similar thou...,False,False
9,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,Some GMO plants have been modified to improve ...,9,gmo plant modify improve nutritional value,False,False


In [ ]:
merge_all_join["row_id"] = merge_all_join.index + 1 #to create sentence level id; the original id is associated article level; 

,id,source,article_final,topic_ng,topic,sentence_each,sentence_turn,sentence_each_lemma,topic_words_YN,topic_words_YN_lead,row_id
0,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,(Family Features) Many consumers are curious t...,0,family feature consumer curious learn purpose ...,False,False,1
1,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,“GMO” is a common term used to describe foods ...,1,gmo common term describe food organism plant m...,False,False,2
2,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,Genetic engineering allows scientists to copy ...,2,genetic engineering allow scientist copy gene ...,False,False,3
3,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,Purpose of GMOs\nHumans have modified crops an...,3,purpose gmos human modify crop animal suit ne...,False,False,4
4,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,"Crossbreeding, selective breeding and mutation...",4,crossbreed selective breeding mutation breedin...,False,False,5
...,...,...,...,...,...,...,...,...,...,...,...
8371,47,Metter Advertiser,"(StatePoint) Starting in early 2020, every chi...",cost,Inflation,The White House recently released a national s...,16,white house recently release national strategy...,False,False,8372
8372,47,Metter Advertiser,"(StatePoint) Starting in early 2020, every chi...",cost,Inflation,"Additionally, a key committee in the House of ...",17,additionally key committee house representativ...,False,False,8373
8373,47,Metter Advertiser,"(StatePoint) Starting in early 2020, every chi...",cost,Inflation,Advocates at the American Heart Association sa...,18,advocate american heart association senate nee...,False,False,8374
8374,47,Metter Advertiser,"(StatePoint) Starting in early 2020, every chi...",cost,Inflation,"To learn more, visit https://act.yourethecure....",19,learn visit httpsactyourethecureorgtqmwpbx,False,False,8375


In [ ]:
# again, find only sentences containing topic words
merge_sentence_select_in = merge_all_join[ (merge_all_join['topic_words_YN']== True)]

merge_sentence_select_in 

,id,source,article_final,topic_ng,topic,sentence_each,sentence_turn,sentence_each_lemma,topic_words_YN,topic_words_YN_lead,row_id
21,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,"As part of this testing, developers consider w...",21,testing developer consider substance add food ...,True,False,22
48,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,"As part of this testing, developers consider w...",21,testing developer consider substance add food ...,True,False,49
75,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,"As part of this testing, developers consider w...",21,testing developer consider substance add food ...,True,False,76
102,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,"As part of this testing, developers consider w...",21,testing developer consider substance add food ...,True,False,103
129,95,Metter Advertiser,(Family Features) Many consumers are curious t...,developer,Affordable housing,"As part of this testing, developers consider w...",21,testing developer consider substance add food ...,True,False,130
...,...,...,...,...,...,...,...,...,...,...,...
8327,72,Metter Advertiser,(BPT) - As a new academic school term begins f...,cost,Inflation,To further help students have equitable access...,13,help student equitable access barne noble coll...,True,False,8328
8334,72,Metter Advertiser,(BPT) - As a new academic school term begins f...,cost,Inflation,The state of student loans\nWhile tuition rate...,20,state student loan tuition rate remain steady...,True,False,8335
8351,15,Metter Advertiser,The annual Communities in Schools Golf Tournam...,cost,Inflation,Cost to play is $75 per person.,3,cost play 75 person,True,False,8352
8355,47,Metter Advertiser,"(StatePoint) Starting in early 2020, every chi...",cost,Inflation,"(StatePoint) Starting in early 2020, every chi...",0,statepoint start early 2020 child united state...,True,False,8356


In [ ]:
merge_sentence_only = merge_sentence_select_in.drop(columns=['article_final'])
merge_sentence_only['str_count'] = merge_sentence_only.sentence_each.str.count(" ") # remove string count less than 3
merge_sentence_only


,id,source,topic_ng,topic,sentence_each,sentence_turn,sentence_each_lemma,topic_words_YN,topic_words_YN_lead,row_id,str_count
21,95,Metter Advertiser,developer,Affordable housing,"As part of this testing, developers consider w...",21,testing developer consider substance add food ...,True,False,22,26
48,95,Metter Advertiser,developer,Affordable housing,"As part of this testing, developers consider w...",21,testing developer consider substance add food ...,True,False,49,26
75,95,Metter Advertiser,developer,Affordable housing,"As part of this testing, developers consider w...",21,testing developer consider substance add food ...,True,False,76,26
102,95,Metter Advertiser,developer,Affordable housing,"As part of this testing, developers consider w...",21,testing developer consider substance add food ...,True,False,103,26
129,95,Metter Advertiser,developer,Affordable housing,"As part of this testing, developers consider w...",21,testing developer consider substance add food ...,True,False,130,26
...,...,...,...,...,...,...,...,...,...,...,...
8327,72,Metter Advertiser,cost,Inflation,To further help students have equitable access...,13,help student equitable access barne noble coll...,True,False,8328,42
8334,72,Metter Advertiser,cost,Inflation,The state of student loans\nWhile tuition rate...,20,state student loan tuition rate remain steady...,True,False,8335,29
8351,15,Metter Advertiser,cost,Inflation,Cost to play is $75 per person.,3,cost play 75 person,True,False,8352,6
8355,47,Metter Advertiser,cost,Inflation,"(StatePoint) Starting in early 2020, every chi...",0,statepoint start early 2020 child united state...,True,False,8356,29


In [ ]:
merge_sentence_only = merge_sentence_only[merge_sentence_only['str_count'] >= 3] #CO: removing short sentences for similar review through Mturk

In [ ]:
merge_sentence_only_10 = merge_sentence_only.groupby(['topic', 'topic_ng']).apply(lambda x: x.sample(10, replace=True)).reset_index(drop=True)
#CO: isn't this also being done above? -> it was from article level; we want the final sample as sentences only; some article contains several sentences with the same topic words


In [ ]:
test = merge_sentence_only_10.groupby(['topic', 'topic_ng'])['id'].count().reset_index()
test 

,topic,topic_ng,id,source,sentence_each,sentence_turn,sentence_each_lemma,topic_words_YN,topic_words_YN_lead,row_id,str_count
0,Affordable housing,developer,10,10,10,10,10,10,10,10,10
1,Affordable housing,poverty,10,10,10,10,10,10,10,10,10
2,Climate change,agriculture,10,10,10,10,10,10,10,10,10
3,Climate change,carbon,10,10,10,10,10,10,10,10,10
4,Crime,gang relate,10,10,10,10,10,10,10,10,10
5,Economy / Jobs,business,10,10,10,10,10,10,10,10,10
6,Economy / Jobs,funding,10,10,10,10,10,10,10,10,10
7,Economy / Jobs,inflation,10,10,10,10,10,10,10,10,10
8,Economy / Jobs,wealth gap,10,10,10,10,10,10,10,10,10
9,Education,educational institution,10,10,10,10,10,10,10,10,10


In [ ]:
merge_sentence_only_10.sample(3)

,id,source,topic_ng,topic,sentence_each,sentence_turn,sentence_each_lemma,topic_words_YN,topic_words_YN_lead,row_id,str_count
216,94,Metter Advertiser,dental care,Health care,"The analysis, “Zocdoc Reports: Dental Download...",3,analysis zocdoc report dental download find ap...,True,False,5901,96
281,61,Metter Advertiser,sanofi,Health care,"La American Lung Association, con el apoyo de ...",4,la american lung association con el apoyo de s...,True,False,7728,27
224,96,Metter Advertiser,hospital,Health care,"Using a laptop, desktop, tablet or mobile devi...",6,laptop desktop tablet mobile device patient ra...,True,True,5993,42


In [ ]:
merge_sentence_only_10_st = merge_sentence_only_10[['id','topic_ng','topic','sentence_each']]
merge_sentence_only_10_st

,id,topic_ng,topic,sentence_each
0,95,developer,Affordable housing,"As part of this testing, developers consider w..."
1,95,developer,Affordable housing,"As part of this testing, developers consider w..."
2,95,developer,Affordable housing,"As part of this testing, developers consider w..."
3,95,developer,Affordable housing,"As part of this testing, developers consider w..."
4,95,developer,Affordable housing,"As part of this testing, developers consider w..."
...,...,...,...,...
305,15,cost,Inflation,Cost to play is $75 per person.
306,72,cost,Inflation,The value of higher education\nRising tuition ...
307,15,cost,Inflation,Cost to play is $75 per person.
308,47,cost,Inflation,"(StatePoint) Starting in early 2020, every chi..."


In [ ]:
# # remove choice

# merge_sentence_only_10_st = merge_sentence_only_10_st[merge_sentence_only_10_st['topic_ng'] != 'choice']

In [ ]:
len(merge_sentence_only_10_st)

310

In [ ]:
merge_sentence_only_10_st = merge_sentence_only_10_st.sample(len(merge_sentence_only_10_st)) #CO: I'm spotting som \n's here. Should that be cleaned out?
merge_sentence_only_10_st #CO: there are two id columns now and they don't match.
#merge_sentence_only_10_st.to_excel('Georgia_topic_words_validation_sentence10_v2.xlsx')

,id,topic_ng,topic,sentence_each
232,46,infectious,Health care,Learn More\nThe Office of Infectious Disease a...
6,95,developer,Affordable housing,"As part of this testing, developers consider w..."
248,22,injury,Health care,Students are reminded to be mindful of other p...
225,21,hospital,Health care,Candler County Hospital will be highlighting b...
35,41,carbon,Climate change,Such innovation centers are focused on catalyz...
...,...,...,...,...
160,75,anxiety depression,Health care,"Additionally, most people with eating disorder..."
121,75,undergrad,Education,"Between newfound freedom, increased workloads,..."
236,46,infectious,Health care,Learn More\nThe Office of Infectious Disease a...
116,74,instruction,Education,Once you’ve picked your favorite out of the bu...
